## Imports

In [2]:
import pandas as pd
import random
import json
from llm_exploration_specifications.data_models.template import Template

## Configurations

In [3]:
FILENAME, DOMAIN, NUMERIC_ATTRIBUTES, ATTRIBUTES, EXCLUDE_ATTRIBUTES, SAMPLE = 'dataset_filename', 'domain', 'numeric_attributes', 'attributes',"exclude_attributes","sample"
dataset_configs = [
    {
        FILENAME : "flights.tsv",
        DOMAIN: "flights",
        NUMERIC_ATTRIBUTES: ['scheduled_trip_time'],
        ATTRIBUTES: [], # fetched dynamically
        EXCLUDE_ATTRIBUTES: [],
        SAMPLE: [] # fetched dynamically
    },
    {
        FILENAME : "netflix.tsv",
        DOMAIN: "shows",
        NUMERIC_ATTRIBUTES: ['duration'],
        ATTRIBUTES: [], # fetched dynamically
        EXCLUDE_ATTRIBUTES: [],
        SAMPLE: [] # fetched dynamically
    },
    {
        FILENAME : "play_store.tsv",
        DOMAIN: "apps",
        NUMERIC_ATTRIBUTES: ['rating', 'reviews', 'app_size_kb', 'install', 'price', 'min_android_ver'],
        ATTRIBUTES: [], # fetched dynamically
        EXCLUDE_ATTRIBUTES: [],
        SAMPLE: [] # fetched dynamically
    }
]
AGG_FUNCTIONS = ['count', 'mean','max','min']
AGGS_TO_STR = {
    "count": "number",
    "mean": "mean",
    "max": "maximum",
    "min": "minimum"
}
NON_NUMERICAL_OPERATIONS = ["eq", "ne", "contains"]
NUMERICAL_OPERATIONS = ["eq","ne","gt","lt"]
OPERATIONS_TO_STR = {
    "eq": "equal",
    "ne": "not equal",
    "contains": "contains",
    "gt": "greater than",
    "lt": "lower than"
}

# Placeholders
DOMAIN_PLACEHOLDER = "<domain>"
ATTRIBUTE_PLACEHOLDER = "<attribute>"
VALUE_PLACEHOLDER = "<value>"
VALUE1_PLACEHOLDER = "<value1>"
VALUE2_PLACEHOLDER = "<value2>"
FILTER_OPERATION_PLACEHOLDER = "<filter_operation>"
AGG_KEY_PLACEHOLDER = "<agg_key>"
AGG_COLUMN_PLACEHOLDER = "<agg_column>"
AGG_FUNC_PLACEHOLDER = "<agg_func>"

GPT35 = "gpt-3.5-turbo"
GPT4 = "gpt-4"

## Templates

In [4]:
templates = [
    Template(
        id = 1,
        placeholders = [DOMAIN_PLACEHOLDER, ATTRIBUTE_PLACEHOLDER],
        text = f"find one {ATTRIBUTE_PLACEHOLDER} of {DOMAIN_PLACEHOLDER} which has one different property compared to the other {ATTRIBUTE_PLACEHOLDER}",
        ldx = f"""
BEGIN CHILDREN {{A1,A2}}
A1 LIKE [F,{ATTRIBUTE_PLACEHOLDER},eq,(?<VALUE>.*)] and CHILDREN {{B1}}
  B1 LIKE [G,(?<COL>.*),(?<AGG_FUNC>.*),(?<AGG_COL>.*)]
A2 LIKE [F,{ATTRIBUTE_PLACEHOLDER},ne,(?<VALUE>.*)] and CHILDREN {{B2}}
  B2 LIKE [G,(?<COL>.*),(?<AGG_FUNC>.*),(?<AGG_COL>.*)]"""
    ), 
     Template(
        id = 2,
        placeholders = [DOMAIN_PLACEHOLDER, ATTRIBUTE_PLACEHOLDER, FILTER_OPERATION_PLACEHOLDER, VALUE_PLACEHOLDER],
        text = f"investigate what makes {DOMAIN_PLACEHOLDER} to have {ATTRIBUTE_PLACEHOLDER} {FILTER_OPERATION_PLACEHOLDER} {VALUE_PLACEHOLDER} and drill down to a specific reason",
        ldx = f"""
BEGIN CHILDREN {{A1}}
A1 LIKE [F,{ATTRIBUTE_PLACEHOLDER},{FILTER_OPERATION_PLACEHOLDER},{VALUE_PLACEHOLDER}] and CHILDREN {{B1,B2}}
  B1 LIKE [G,(?<COL1>.*),.*]
  B2 LIKE [F,(?<COL1>.*),eq,.*] and CHILDREN {{C1, C2}}
      C1 LIKE [G,(?<COL2>.*),.*]
      C2 LIKE [F,(?<COL2>.*),eq,.*]"""
     ),
    Template(
        id = 3,
        placeholders = [DOMAIN_PLACEHOLDER, ATTRIBUTE_PLACEHOLDER],
        text = f"compare some three subsets of {ATTRIBUTE_PLACEHOLDER} according to some properties",
        ldx = f"""
BEGIN CHILDREN {{A1,A2,A3}}
A1 LIKE [F,{ATTRIBUTE_PLACEHOLDER},eq,.*] and CHILDREN {{B1}}
    B1 LIKE [G,(?<COL>.*),(?<AGG_FUNC>.*),(?<AGG_COL>.*)]
A2 LIKE [F,{ATTRIBUTE_PLACEHOLDER},eq,.*] and CHILDREN {{B2}}
    B2 LIKE [G,(?<COL>.*),(?<AGG_FUNC>.*),(?<AGG_COL>.*)]
A3 LIKE [F,{ATTRIBUTE_PLACEHOLDER},eq,.*] and CHILDREN {{B3}}
    B3 LIKE [G,(?<COL>.*),(?<AGG_FUNC>.*),(?<AGG_COL>.*)]"""
    ),
    Template(
        id = 4,
        placeholders = [DOMAIN_PLACEHOLDER, AGG_FUNC_PLACEHOLDER, AGG_COLUMN_PLACEHOLDER],
        text = f"show {AGG_FUNC_PLACEHOLDER} of {AGG_COLUMN_PLACEHOLDER} of two different subsets of {DOMAIN_PLACEHOLDER}",
        ldx = f"""
BEGIN CHILDREN {{A1,A2}}
A1 LIKE [F,.*,eq,.*] and CHILDREN {{B1}}
    B1 LIKE [G,.*,{AGG_FUNC_PLACEHOLDER},{AGG_COLUMN_PLACEHOLDER}]
A2 LIKE [F,.*,eq,.*] and CHILDREN {{B2}}
    B2 LIKE [G,.*,{AGG_FUNC_PLACEHOLDER},{AGG_COLUMN_PLACEHOLDER}]"""
    ), 
    Template(
        id = 5,
        placeholders = [DOMAIN_PLACEHOLDER, ATTRIBUTE_PLACEHOLDER, FILTER_OPERATION_PLACEHOLDER, VALUE_PLACEHOLDER],
        text = f"show two properties of {DOMAIN_PLACEHOLDER} with {ATTRIBUTE_PLACEHOLDER} {FILTER_OPERATION_PLACEHOLDER} {VALUE_PLACEHOLDER} compared to the global {DOMAIN_PLACEHOLDER}",
        ldx = f"""
BEGIN CHILDREN {{A1,A2,A3}}
A1 LIKE [G,(?<COL1>.*),(?<AGG_FUNC1>.*),(?<AGG_COL1>.*)]
A2 LIKE [G,(?<COL2>.*),(?<AGG_FUNC2>.*),(?<AGG_COL2>.*)]
A3 LIKE [F,{ATTRIBUTE_PLACEHOLDER},{FILTER_OPERATION_PLACEHOLDER},{VALUE_PLACEHOLDER}] and CHILDREN {{B1,B2}}
    B1 LIKE [G,(?<COL1>.*),(?<AGG_FUNC1>.*),(?<AGG_COL1>.*)]
    B2 LIKE [G,(?<COL2>.*),(?<AGG_FUNC2>.*),(?<AGG_COL2>.*)]"""
    ), 
    Template(
        id = 6,
        placeholders = [DOMAIN_PLACEHOLDER, ATTRIBUTE_PLACEHOLDER],
        text = f"explore in different ways three different {ATTRIBUTE_PLACEHOLDER} of {DOMAIN_PLACEHOLDER}",
        ldx = f"""
BEGIN CHILDREN {{A1,A2,A3}}
A1 LIKE [F,{ATTRIBUTE_PLACEHOLDER},eq,.*] and CHILDREN {{B1}}
    B1 LIKE [G,.*]
A2 LIKE [F,{ATTRIBUTE_PLACEHOLDER},eq,.*] and CHILDREN {{B2}}
    B2 LIKE [G,.*]
A3 LIKE [F,{ATTRIBUTE_PLACEHOLDER},eq,.*] and CHILDREN {{B3}}
    B3 LIKE [G,.*]"""
    ),
    Template(
        id = 8,
        placeholders = [DOMAIN_PLACEHOLDER, ATTRIBUTE_PLACEHOLDER, VALUE_PLACEHOLDER, FILTER_OPERATION_PLACEHOLDER],
        text = f"explore the data, make sure to address two interesting aspects of {DOMAIN_PLACEHOLDER} with {ATTRIBUTE_PLACEHOLDER} {FILTER_OPERATION_PLACEHOLDER} {VALUE_PLACEHOLDER}",
        ldx = f"""
BEGIN DESCENDANTS {{A1}}
A1 LIKE [F,{ATTRIBUTE_PLACEHOLDER},{FILTER_OPERATION_PLACEHOLDER},{VALUE_PLACEHOLDER}] and CHILDREN {{B1,B2}}
    B1 LIKE [G,.*]
    B2 LIKE [G,.*]"""
    ),
    Template(
        id = 9,
        placeholders = [DOMAIN_PLACEHOLDER, ATTRIBUTE_PLACEHOLDER, FILTER_OPERATION_PLACEHOLDER, VALUE_PLACEHOLDER],
        text = f"show some interesting sub-groups (groups within groups) of {DOMAIN_PLACEHOLDER} with {ATTRIBUTE_PLACEHOLDER} {FILTER_OPERATION_PLACEHOLDER} {VALUE_PLACEHOLDER}",
        ldx = f"""
BEGIN CHILDREN {{A1}}
A1 LIKE [F,{ATTRIBUTE_PLACEHOLDER},{FILTER_OPERATION_PLACEHOLDER},{VALUE_PLACEHOLDER}] and CHILDREN {{B1}}
    B1 LIKE [G,.*] and CHILDREN {{C1}}
        C1 LIKE [G,.*]"""
    )
]

In [5]:
print(f'# of templates families is: {len(templates)}')
templates = [template for template in templates if template.enabled]
print(f'# of enabled templates is: {len(templates)}')

# of templates families is: 8
# of enabled templates is: 8


## Generate Populated Templates

In [6]:
def populate_placeholder(df, dataset_config, placeholder, column=None):
    global last_agg_func
    global last_filter_att
    if placeholder == DOMAIN_PLACEHOLDER:
        return dataset_config[DOMAIN]
    if placeholder == ATTRIBUTE_PLACEHOLDER:
        last_filter_att = random.choice(dataset_config[ATTRIBUTES])
        return last_filter_att
    if placeholder.startswith("<value"):
        return str(df.loc[:,column].sample().array[0])
    if placeholder == FILTER_OPERATION_PLACEHOLDER:
        operations = NUMERICAL_OPERATIONS if last_filter_att in dataset_config[NUMERIC_ATTRIBUTES] else NON_NUMERICAL_OPERATIONS
        return random.choice(operations)
    if placeholder == AGG_KEY_PLACEHOLDER:
        return random.choice(list(set(dataset_config[ATTRIBUTES]) - set(dataset_config[NUMERIC_ATTRIBUTES])))
    if placeholder == AGG_COLUMN_PLACEHOLDER:
        if last_agg_func != "COUNT":
            return random.choice(dataset_config[NUMERIC_ATTRIBUTES])
        else:
            return random.choice(list(set(dataset_config[ATTRIBUTES]) - set(dataset_config[NUMERIC_ATTRIBUTES])))
    if placeholder == AGG_FUNC_PLACEHOLDER:
        func = random.choice(AGG_FUNCTIONS)
        last_agg_func = func
        return func

def create_queries_from_template(dataset_config, df, template: Template):
    new_pairs = set()
    placeholders = template.placeholders
    ldx = template.ldx
    text = template.text
    _id = template.id

    # choose values for placeholders
    placeholder2value = {}
    for placeholder in placeholders:
        if placeholder.startswith("<value"):
            chosen_attribute = placeholder2value[ATTRIBUTE_PLACEHOLDER]
            value = populate_placeholder(df, dataset_config, placeholder, column=chosen_attribute)
        else:
            value = populate_placeholder(df, dataset_config, placeholder)
        placeholder2value[placeholder] = value

    # replace placeholders
    for placeholder,value in placeholder2value.items():
        ldx = ldx.replace(placeholder,value)
        value_text = value if placeholder != FILTER_OPERATION_PLACEHOLDER else OPERATIONS_TO_STR[value]
        value_text = value_text if placeholder != AGG_FUNC_PLACEHOLDER else AGGS_TO_STR[value]
        text = text.replace(placeholder,value_text)

    # new_pairs.add((_id,text,query.replace('\n','').strip()))
    new_pairs.add((_id,text,ldx.strip(),','.join(dataset_config[ATTRIBUTES]),dataset_config[FILENAME].split("/")[-1]))
    return new_pairs

In [8]:
# create manipulated queries from templates N times
N = 10
test_dataset = []

for dataset_config in dataset_configs:
    df = pd.read_csv("../datasets/" + dataset_config[FILENAME], sep='\t', header=0)
    dataset_config[ATTRIBUTES] = list(df.columns)[1:]
    dataset_config[ATTRIBUTES] = [item for item in dataset_config[ATTRIBUTES] if item not in dataset_config[EXCLUDE_ATTRIBUTES]]
    dataset_config[SAMPLE] = df.drop(columns=['Unnamed: 0']).head().to_string(index=False, justify='left')
    
    for i,template in enumerate(templates):
        template_examples = set()
        for j in range(N):
            template_examples.update(create_queries_from_template(dataset_config, df, template))
        test_dataset.extend(list(template_examples))    

# show amount how data for each template
d_test = {}
for task in test_dataset:
    d_test[task[0]] = d_test.get(task[0],0)+1
            
print(d_test)
print(sum(d_test.values()))

{1: 22, 2: 28, 3: 21, 4: 15, 5: 30, 6: 23, 8: 30, 9: 30}
199


In [16]:
with open("NL2LDX-benchmark.json", "w") as fp:   #Pickling
    json.dump(test_dataset, fp)

## Paraphrase using OpenAI

In [17]:
# from exploration_plan_generator.clients.openai_client import OpenAIClient
# 
# # Paraphrasing
# test_dataset_paraphrased = []
# c = OpenAIClient(GPT35)
# 
# for id, text, ldx_expected, scheme, dataset in test_dataset:
#     print("before paraphrasing: " + text)
#     text_paraphrased = c.send_request("", "", f"paraphrase the following task without changing its meaning and make the attributes human readable: {text}", should_print=False)
#     test_dataset_paraphrased.append((id, text_paraphrased, ldx_expected, scheme, dataset))
# 
# with open("NL2LDX-benchmark.json", "w") as fp:
#     json.dump(test_dataset_paraphrased, fp)

## Manually fix & paraphrase